<a href="https://colab.research.google.com/github/soohyunme/TensorFlow_Tutorial/blob/main/Code/08_Custom_Layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

# Device setting

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0],True)

# Load data

In [3]:
(x_train,y_train), (x_test,y_test) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


# Normalize

In [4]:
x_train = x_train.reshape(-1, 28 * 28).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28 * 28).astype('float32') / 255.0

#Model

In [5]:
class Dense(layers.Layer):
  def __init__(self, units):
    super(Dense, self).__init__()
    self.units = units

  def build(self, input_shape):
    self.w = self.add_weight(
        name = 'w',
        shape = (input_shape[-1], self.units),
        initializer = 'random_normal',
        trainable = True,
    )

    self.b = self.add_weight(
        name = 'b',
        shape = (self.units,),
        initializer = 'zeros',
        trainable = True,
    )

  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b

In [6]:
class MyReLU(layers.Layer):
  def __init__(self):
    super(MyReLU, self).__init__()

  def call(self, x):
    return tf.math.maximum(x, 0)
    

In [7]:
class MyModel(keras.Model):
  def __init__(self, num_classes=10):
    super(MyModel, self).__init__()
    self.dense1 = Dense(64)
    self.dense2 = Dense(num_classes)
    self.relu = MyReLU()

    # self.dense1 = layers.Dense(64)
    # self.dense2 = layers.Dense(num_classes)

  def call(self, input_tensor):
    x = self.relu(self.dense1(input_tensor))
    
    # x = tf.nn.relu(self.dense1(input_tensor))
    
    return self.dense2(x)


In [8]:
model = MyModel()
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(),
    metrics = ['accuracy'],
)
model.fit(x_train, y_train, batch_size=32, epochs=2, verbose=2)
model.evaluate(x_test, y_test, batch_size=32, verbose=2)


Epoch 1/2
1875/1875 - 10s - loss: 0.3430 - accuracy: 0.9054 - 10s/epoch - 5ms/step
Epoch 2/2
1875/1875 - 6s - loss: 0.1647 - accuracy: 0.9520 - 6s/epoch - 3ms/step
313/313 - 1s - loss: 0.1389 - accuracy: 0.9583 - 809ms/epoch - 3ms/step


[0.13891787827014923, 0.958299994468689]